In [1]:
import os
import sys
sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"

import quantfreedom as qf
import pandas as pd
import numpy as np
import numba
from quantfreedom.backtester.base.base import rsi_below_entries, log_trades_plot
from quantfreedom.backtester.nb.simulate import simulate_from_signals
from quantfreedom.backtester.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
    or_dt,
    cart_array_dt,
)
pd.options.display.float_format = '{:,.2f}'.format
# qf.clear_cache()

prices = pd.read_csv('30min.csv', index_col='time')
entries = rsi_below_entries(
    timeperiods=[20],
    below_ranges=[50],
    prices=prices.close
)


In [2]:
simulate_from_signals(
    open_prices=prices.open.values,
    high_prices=prices.high.values,
    low_prices=prices.low.values,
    close_prices=prices.close.values,
    entries=entries.values,
    equity=1000.,
    fee_pct=.06,
    mmr=.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=3.,
    risk_rewards=[3,4,5],
    size_pct=1.,
    tsl_pcts_init=[3, 4, 5],
    tsl_true_or_false=True,
    tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    tsl_trail_by_pct=1,
    tsl_when_pct_from_avg_entry=[3,4,5],
    gains_pct_filter=63,
)


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mType of variable 'cart_array.146' cannot be determined, operation: unknown operation, location: unknown location (0:0)
[1m
File "unknown location", line 0:[0m
[1m<source missing, REPL/exec in use?>[0m
[0m

In [3]:
df_or = pd.DataFrame(order_rec)
for i in range(len(OrderType._fields)):
    df_or.replace({'order_type': {i: OrderType._fields[i]}}, inplace=True)
df_or.head(20)
    

NameError: name 'order_rec' is not defined

In [ ]:
pd.DataFrame(df)


In [ ]:
df_cart = pd.DataFrame(cart).dropna(axis='columns', thresh=1)
for i in range(len(SL_BE_or_Trail_BasedOn._fields)):
    df_cart.replace({'tsl_based_on': {i: SL_BE_or_Trail_BasedOn._fields[i]}}, inplace=True)
    df_cart.replace({'sl_to_be_based_on': {i: SL_BE_or_Trail_BasedOn._fields[i]}}, inplace=True)
df_cart.T

In [ ]:
log_trades_plot(
    open_prices=prices.open,
    high_prices=prices.high,
    low_prices=prices.low,
    close_prices=prices.close,
    order_records=order_rec[3:15],
)
